In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mrdog123","key":"35288ba7227552ae60e10a140227ad66"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 97% 321M/331M [00:03<00:00, 113MB/s]
100% 331M/331M [00:03<00:00, 94.7MB/s]


In [5]:
'chmod 600 /root/.kaggle/kaggle.json'

'chmod 600 /root/.kaggle/kaggle.json'

In [6]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [7]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation


In [8]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [9]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [10]:
len(age)

23708

In [11]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [12]:
df.head()

,age,gender,img
0,78,0,78_0_0_20170111222218304.jpg.chip.jpg
1,45,0,45_0_0_20170117192031378.jpg.chip.jpg
2,20,0,20_0_1_20170116204110339.jpg.chip.jpg
3,1,1,1_1_3_20161220220534186.jpg.chip.jpg
4,24,0,24_0_2_20170116170804180.jpg.chip.jpg


In [13]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
train_dataset = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_dataset = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')


Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [16]:
IMG_SIZE = (200,200,3) ;

In [17]:
from keras.initializers import he_uniform

In [18]:
input_shape=(200, 200 , 3)
base_model = keras.applications.ResNet50(
  include_top=False ,
  weights="imagenet",
  input_shape=(200, 200 , 3))
base_model.trainable = True ;
fine_tune_at = 130 ;
for layer in base_model.layers[:fine_tune_at]  :
    layer.trainable = False ;
output = base_model.layers[-1].output
output = tfl.GlobalAveragePooling2D()(output)
output = tfl.Dropout(0.2)(output)
flatten = Flatten()(output)

dense1 = Dense(512, activation='relu', kernel_initializer=he_uniform())(flatten)
dense2 = Dense(512,activation='relu', kernel_initializer=he_uniform())(flatten)

dense3 = Dense(512,activation='relu',kernel_initializer=he_uniform())(dense1)
dense4 = Dense(512,activation='relu',kernel_initializer=he_uniform())(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

94765736/94765736 [==============================] - 0s 0us/step


In [19]:
model = Model(inputs=base_model.input,outputs=[output1,output2])

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [21]:
model.fit(train_dataset, batch_size=32, epochs=7, validation_data=test_dataset)

Epoch 1/7
625/625 [==============================] - 276s 409ms/step - loss: 78.6860 - age_loss: 14.8936 - gender_loss: 0.6444 - age_mae: 14.8936 - gender_accuracy: 0.6287 - val_loss: 77.2689 - val_age_loss: 22.7998 - val_gender_loss: 0.5502 - val_age_mae: 22.7998 - val_gender_accuracy: 0.7206
Epoch 2/7
625/625 [==============================] - 251s 401ms/step - loss: 74.2990 - age_loss: 14.3100 - gender_loss: 0.6059 - age_mae: 14.3100 - gender_accuracy: 0.6790 - val_loss: 166.3571 - val_age_loss: 38.5965 - val_gender_loss: 1.2905 - val_age_mae: 38.5965 - val_gender_accuracy: 0.5062
Epoch 3/7
625/625 [==============================] - 254s 407ms/step - loss: 73.1821 - age_loss: 14.0819 - gender_loss: 0.5970 - age_mae: 14.0819 - gender_accuracy: 0.6851 - val_loss: 119.6459 - val_age_loss: 23.6569 - val_gender_loss: 0.9696 - val_age_mae: 23.6569 - val_gender_accuracy: 0.5167
Epoch 4/7
625/625 [==============================] - 249s 397ms/step - loss: 72.1330 - age_loss: 14.0044 - gender

In [ ]:
keras.src.callbacks.History

keras.src.callbacks.History